In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow
import keras
import imutils
from imutils import paths
from skimage import transform
from skimage import exposure
from skimage import io
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
import random
import os
import warnings
warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
class TrafficSignNet:
    def build(width,height,depth,classes):
        model=Sequential()
        inputShape=(height,width,depth)
        chanDim=-1
        
        model.add(Conv2D(8,(5,5),activation='relu',padding='same',input_shape=inputShape))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(Conv2D(16,(3,3),activation='relu',padding='same'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(16,(3,3),activation='relu',padding='same'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        
        model.add(Flatten())
        model.add(Dense(128,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        model.add(Dense(256,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes,activation='softmax'))
        return model

In [3]:
def load_split(basePath,csvPath):
    data=[]
    labels=[]
    
    rows=open(csvPath).read().strip().split('\n')[1:]
    random.shuffle(rows)
    
    for (i,row) in enumerate(rows):
        if i>0 and i%1000==0:
            print("[INFO] processed {} total images".format(i))
        (label,imagePath)=row.strip().split(',')[-2:]
        imagePath=os.path.sep.join([basePath,imagePath])
        image=io.imread(imagePath)
        image=transform.resize(image,(32,32))
        image=exposure.equalize_adapthist(image,clip_limit=0.1)
        
        data.append(image)
        labels.append(label)
        
    data=np.array(data)
    labels=np.array(labels)
    
    return (data,labels)

In [4]:
args={'dataset':r"F:\Computer Vision Datasets\Traffic Sign Detection\archive (7)"}

In [5]:
labelNames=open('Meta.csv').read().strip().split('\n')[1:]
labelNames=[l.split(",")[1] for l in labelNames]

In [6]:
trainPath=os.path.sep.join([args['dataset'],'Train.csv'])
testPath=os.path.sep.join([args['dataset'],'Test.csv'])

(trainX,trainY)=load_split(args['dataset'],trainPath)
(testX,testY)=load_split(args['dataset'],testPath)

trainX=trainX.astype('float32')/255.0
testX=testX.astype('float32')/255.0

numLabels=len(np.unique(trainY))
trainY=to_categorical(trainY,numLabels)
testY=to_categorical(testY,numLabels)

classTotals=trainY.sum(axis=0)
classWeight=dict()

for i in range(len(classTotals)):
    classWeight[i]=classTotals.max()/classTotals[i]

[INFO] processed 1000 total images
[INFO] processed 2000 total images
[INFO] processed 3000 total images
[INFO] processed 4000 total images
[INFO] processed 5000 total images
[INFO] processed 6000 total images
[INFO] processed 7000 total images
[INFO] processed 8000 total images
[INFO] processed 9000 total images
[INFO] processed 10000 total images
[INFO] processed 11000 total images
[INFO] processed 12000 total images
[INFO] processed 13000 total images
[INFO] processed 14000 total images
[INFO] processed 15000 total images
[INFO] processed 16000 total images
[INFO] processed 17000 total images
[INFO] processed 18000 total images
[INFO] processed 19000 total images
[INFO] processed 20000 total images
[INFO] processed 21000 total images
[INFO] processed 22000 total images
[INFO] processed 23000 total images
[INFO] processed 24000 total images
[INFO] processed 25000 total images
[INFO] processed 26000 total images
[INFO] processed 27000 total images
[INFO] processed 28000 total images
[

In [7]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
aug=ImageDataGenerator(rotation_range=10,
	zoom_range=0.15,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")

Num_epochs=20
lr=1e-3
bs=64

opt=Adam(lr=lr,decay=Num_epochs/lr)
model=TrafficSignNet.build(width=32,height=32,depth=3,classes=numLabels)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

H=model.fit(aug.flow(trainX,trainY,batch_size=bs),
            validation_data=(testX,testY),
            steps_per_epoch=trainX.shape[0]//bs,
            epochs=Num_epochs,
            class_weight=classWeight,
            verbose=1)

Train for 612 steps, validate on 12630 samples
Epoch 1/20
612/612 [==============================] - 87s 142ms/step - loss: 5.2103 - accuracy: 0.0244 - val_loss: 10.4523 - val_accuracy: 0.0217
Epoch 2/20
612/612 [==============================] - 80s 131ms/step - loss: 5.2083 - accuracy: 0.0254 - val_loss: 10.2381 - val_accuracy: 0.0345
Epoch 3/20
612/612 [==============================] - 86s 141ms/step - loss: 5.1889 - accuracy: 0.0264 - val_loss: 10.2544 - val_accuracy: 0.0352
Epoch 4/20
612/612 [==============================] - 83s 136ms/step - loss: 5.2038 - accuracy: 0.0259 - val_loss: 10.2572 - val_accuracy: 0.0356
Epoch 5/20
612/612 [==============================] - 79s 130ms/step - loss: 5.2128 - accuracy: 0.0246 - val_loss: 10.2530 - val_accuracy: 0.0353
Epoch 6/20
612/612 [==============================] - 79s 129ms/step - loss: 5.2072 - accuracy: 0.0252 - val_loss: 10.2483 - val_accuracy: 0.0350
Epoch 7/20
612/612 [==============================] - 78s 127ms/step - loss: 

In [9]:
from numpy import argmax
predictions=model.predict(testX,batch_size=bs)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1),target_names=labelNames))

              precision    recall  f1-score   support

          27       0.00      0.00      0.00        60
           0       0.12      0.18      0.14       720
           1       0.06      0.01      0.02       750
          10       0.03      0.01      0.02       450
          11       0.06      0.05      0.06       660
          12       0.05      0.03      0.04       630
          13       0.00      0.00      0.00       150
          14       0.04      0.03      0.04       450
          15       0.04      0.01      0.01       450
          16       0.06      0.04      0.05       480
          17       0.04      0.06      0.05       660
          18       0.04      0.05      0.04       420
          19       0.04      0.01      0.01       690
           2       0.06      0.02      0.03       720
          20       0.03      0.02      0.02       270
          21       0.04      0.05      0.04       210
          22       0.00      0.00      0.00       150
          23       0.39    

In [10]:
model.save('TrafficSignClassification.h5')

In [11]:
from tensorflow.keras.models import load_model
my_model=load_model('TrafficSignClassification.h5')

In [12]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 8)         608       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 8)         32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 16)        6

In [13]:
args={'test_set':r"F:\Computer Vision Datasets\Traffic Sign Detection\archive (7)\Test",
     'output_directory':r"F:\Computer Vision Datasets\Traffic Sign Detection\Testing Directory"}

In [16]:
labelNames=open('Meta.csv').read().strip().split("\n")[1:]
labelNames=[l.split(",")[1] for l in labelNames]

In [17]:
imagePaths=list(paths.list_images(args['test_set']))
random.shuffle(imagePaths)
imagePaths=imagePaths[:25]

In [18]:
for (i,imagePath) in enumerate(imagePaths):
    image=io.imread(imagePath)
    image=transform.resize(image,(32,32))
    image=exposure.equalize_adapthist(image,clip_limit=0.1)
    
    image=image.astype('float32')/255.0
    image=np.expand_dims(image,axis=0)
    
    preds=my_model.predict(image)
    j=preds.argmax(axis=1)[0]
    label=labelNames[j]
    
    image=cv2.imread(imagePath)
    image=imutils.resize(image,width=128)
    cv2.putText(image,label,(5,15),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),2)
    
    p=os.path.sep.join([args['output_directory'],"{}.png".format(i)])
    cv2.imwrite(p,image)